# 💾 Redis como Caché con MongoDB - Demo

*Autor: @mCárdenas 2025*

Demostración práctica del uso de Redis como capa de caché.

In [ ]:
import redis
from pymongo import MongoClient
import json
import time
from datetime import datetime

# Conexiones
r = redis.Redis(host='localhost', port=6379, decode_responses=True)
mongo = MongoClient('mongodb://localhost:27017/')
db = mongo['tienda_demo']

print(f"✅ Redis: {r.ping()}")
print(f"✅ MongoDB: {db.command('ping')}")

## 1. Preparar Datos de Prueba

In [ ]:
# Limpiar datos anteriores
db.productos.delete_many({})
r.flushdb()

# Insertar productos de prueba
productos = [
    {"_id": "P001", "nombre": "Laptop Gaming", "precio": 1299.99, "stock": 50},
    {"_id": "P002", "nombre": "Monitor 4K", "precio": 499.99, "stock": 100},
    {"_id": "P003", "nombre": "Teclado Mecánico", "precio": 149.99, "stock": 200},
    {"_id": "P004", "nombre": "Mouse Inalámbrico", "precio": 79.99, "stock": 300},
    {"_id": "P005", "nombre": "Auriculares Pro", "precio": 299.99, "stock": 75},
]

db.productos.insert_many(productos)
print(f"✅ Insertados {len(productos)} productos en MongoDB")

## 2. Implementar Cache-Aside Pattern

In [ ]:
class ProductoCache:
    """Repositorio con patrón Cache-Aside."""
    
    def __init__(self, redis_client, mongo_collection, ttl=3600):
        self.r = redis_client
        self.db = mongo_collection
        self.ttl = ttl
        self.hits = 0
        self.misses = 0
    
    def _cache_key(self, producto_id):
        return f"cache:producto:{producto_id}"
    
    def obtener(self, producto_id):
        """Obtener producto con caché."""
        cache_key = self._cache_key(producto_id)
        
        # 1. Buscar en caché
        cached = self.r.get(cache_key)
        if cached:
            self.hits += 1
            return json.loads(cached), "HIT"
        
        # 2. Cache MISS - buscar en MongoDB
        self.misses += 1
        producto = self.db.find_one({"_id": producto_id})
        
        if producto:
            # 3. Guardar en caché
            self.r.setex(cache_key, self.ttl, json.dumps(producto))
        
        return producto, "MISS"
    
    def actualizar(self, producto_id, datos):
        """Actualizar e invalidar caché."""
        self.db.update_one({"_id": producto_id}, {"$set": datos})
        self.r.delete(self._cache_key(producto_id))  # Invalidar
        return self.obtener(producto_id)
    
    def stats(self):
        total = self.hits + self.misses
        ratio = self.hits / total * 100 if total > 0 else 0
        return {"hits": self.hits, "misses": self.misses, "hit_ratio": f"{ratio:.1f}%"}

# Crear instancia
cache = ProductoCache(r, db.productos, ttl=60)
print("✅ Cache inicializado")

## 3. Demostrar Cache Hit vs Miss

In [ ]:
print("📊 Demostración de Cache Hit/Miss\n")

# Primera consulta: MISS (va a MongoDB)
start = time.time()
producto, status = cache.obtener("P001")
tiempo1 = (time.time() - start) * 1000
print(f"1ª consulta: {status} - {tiempo1:.3f}ms")
print(f"   Producto: {producto['nombre']}\n")

# Segunda consulta: HIT (viene de Redis)
start = time.time()
producto, status = cache.obtener("P001")
tiempo2 = (time.time() - start) * 1000
print(f"2ª consulta: {status} - {tiempo2:.3f}ms")
print(f"   Producto: {producto['nombre']}\n")

# Comparación
if tiempo1 > 0:
    mejora = tiempo1 / tiempo2 if tiempo2 > 0 else float('inf')
    print(f"⚡ Cache es {mejora:.1f}x más rápido")

print(f"\n📈 Estadísticas: {cache.stats()}")

## 4. Simular Carga de Trabajo

In [ ]:
import random

# Reset stats
cache.hits = 0
cache.misses = 0

print("🔄 Simulando 100 consultas aleatorias...\n")

ids = ["P001", "P002", "P003", "P004", "P005"]
tiempos = []

for i in range(100):
    producto_id = random.choice(ids)
    start = time.time()
    producto, status = cache.obtener(producto_id)
    tiempos.append(time.time() - start)

# Estadísticas
stats = cache.stats()
promedio = sum(tiempos) / len(tiempos) * 1000

print(f"📊 Resultados:")
print(f"   Hits: {stats['hits']}")
print(f"   Misses: {stats['misses']}")
print(f"   Hit Ratio: {stats['hit_ratio']}")
print(f"   Tiempo promedio: {promedio:.3f}ms")

## 5. Invalidación de Caché

In [ ]:
print("🔄 Demostración de invalidación de caché")

# Obtener producto (debería estar en caché)
producto, status = cache.obtener("P001")
print(f"Antes de actualizar: {status}")
print(f"  Precio: ${producto['precio']}")

# Actualizar precio (invalida caché automáticamente)
cache.actualizar("P001", {"precio": 999.99})
print("✏️ Precio actualizado a $999.99")

# La siguiente consulta será un MISS porque el caché se invalidó
producto, status = cache.obtener("P001")
print(f"Después de actualizar: {status}")
print(f"  Precio: ${producto['precio']}")

## 6. TTL (Time To Live)

In [ ]:
# Crear caché con TTL muy corto para demostración
cache_corto = ProductoCache(r, db.productos, ttl=3)  # 3 segundos

print("⏱️ Demostración de TTL (3 segundos)\n")

# Primera consulta
producto, status = cache_corto.obtener("P002")
print(f"T=0s: {status}")

# Consulta inmediata
producto, status = cache_corto.obtener("P002")
print(f"T=0s: {status}")

# Esperar a que expire
print("⏳ Esperando 4 segundos...\n")
time.sleep(4)

# Consulta después de expirar
producto, status = cache_corto.obtener("P002")
print(f"T=4s: {status} (caché expirado)")

## 7. Comparación de Rendimiento

In [ ]:
import statistics

def consulta_sin_cache(producto_id, n=50):
    """Consultar directamente a MongoDB."""
    tiempos = []
    for _ in range(n):
        start = time.time()
        db.productos.find_one({"_id": producto_id})
        tiempos.append((time.time() - start) * 1000)
    return tiempos

def consulta_con_cache(producto_id, n=50):
    """Consultar con caché (después del primer miss)."""
    cache.obtener(producto_id)  # Primer miss
    tiempos = []
    for _ in range(n):
        start = time.time()
        cache.obtener(producto_id)
        tiempos.append((time.time() - start) * 1000)
    return tiempos

print("📊 Comparación de rendimiento (50 consultas)")

# Sin caché
tiempos_mongo = consulta_sin_cache("P003")
avg_mongo = statistics.mean(tiempos_mongo)
print(f"MongoDB directo:")
print(f"  Promedio: {avg_mongo:.3f}ms")
print(f"  Min: {min(tiempos_mongo):.3f}ms")
print(f"  Max: {max(tiempos_mongo):.3f}ms")

# Con caché
tiempos_cache = consulta_con_cache("P003")
avg_cache = statistics.mean(tiempos_cache)
print(f"Redis caché:")
print(f"  Promedio: {avg_cache:.3f}ms")
print(f"  Min: {min(tiempos_cache):.3f}ms")
print(f"  Max: {max(tiempos_cache):.3f}ms")

# Mejora
if avg_cache > 0:
    mejora = avg_mongo / avg_cache
    print(f"⚡ Redis es {mejora:.1f}x más rápido que MongoDB directo")

## 8. Limpiar

In [ ]:
# Limpiar datos de demo
# db.productos.delete_many({})
# r.flushdb()
# print("✅ Datos de demo limpiados")

print(f"📊 Estadísticas finales del caché: {cache.stats()}")

## 📚 Resumen

| Aspecto | Sin Caché | Con Caché |
|---------|-----------|----------|
| Latencia | 5-50ms | <1ms |
| Carga en DB | Alta | Baja |
| Complejidad | Baja | Media |
| Consistencia | Siempre actual | Puede estar desactualizado |